In [1]:
import pandas as pd
import os
import re

import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import (
    SemanticSplitterNodeParser,
)
from transformers import GPT2Tokenizer
from llama_index.core import Document
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/isaiaszc/pathway/pathway-
[nltk_data]     indexer/.venv/lib/python3.12/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!
/home/isaiaszc/pathway/pathway-indexer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datapath = "../../data"


In [3]:
embed_model_name = "text-embedding-3-large"

embed_model = OpenAIEmbedding(
    model=embed_model_name,
    embed_batch_size=10,
    max_retries=10,
    timeout=180,
    reuse_client=False,
)


In [4]:
chroma_client = chromadb.EphemeralClient()
# delete collection if it exists
if any(coll.name == "test" for coll in chroma_client.list_collections()):
    chroma_client.delete_collection("test")
chroma_collection = chroma_client.create_collection("test")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


In [5]:
# SEMANTIC SPLITTER
buffer_size = 1
splitter = SemanticSplitterNodeParser(
    buffer_size=buffer_size,
    breakpoint_percentile_threshold=81,
    include_prev_next_rel=False,
    embed_model=embed_model,
)


In [6]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


def split_text_by_tokens(text, max_tokens=8000):
    """Divide the text into chunks that do not exceed the maximum number of tokens"""
    words = text.split()
    chunks = []
    chunk = []
    tokens_count = 0

    for word in words:
        tokens_count += len(tokenizer.tokenize(word))
        if tokens_count >= max_tokens:
            chunks.append(" ".join(chunk))
            chunk = []
            tokens_count = len(tokenizer.tokenize(word))
        chunk.append(word)

    if chunk:
        chunks.append(" ".join(chunk))
    return chunks


/home/isaiaszc/pathway/pathway-indexer/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# create a simple ingestion pipeline: chunk the documents and create embeddings
pipeline = IngestionPipeline(
    transformations=[
        splitter,
        embed_model,
    ]
)


In [8]:
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)


## Load Files


In [9]:
origin_paths = [
    f"{datapath}/data_09_12_24/out_sep_12/from_html/",
    f"{datapath}/data_09_12_24/out_sep_12/from_pdf/",
]

# Read the document names from the directories:
files_list = [path + item for path in origin_paths for item in os.listdir(path)]

# drop txt files
files_list = [item for item in files_list if not item.endswith(".txt")]

files_list.sort()
files_list[0:10]


['../../data/data_09_12_24/out_sep_12/from_html/-Admission-Requirements.md',
 '../../data/data_09_12_24/out_sep_12/from_html/-After-PathwayConnect.md',
 '../../data/data_09_12_24/out_sep_12/from_html/-Answers-to-Your-Questions-about-Ecclesiastical-Endorsement.md',
 '../../data/data_09_12_24/out_sep_12/from_html/-Application-Process.md',
 '../../data/data_09_12_24/out_sep_12/from_html/-Assistance-for-Students-with-Disabilities.md',
 '../../data/data_09_12_24/out_sep_12/from_html/-BYU-Idaho-Course-Exceptions.md',
 '../../data/data_09_12_24/out_sep_12/from_html/-BYU-Pathway-Support.md',
 '../../data/data_09_12_24/out_sep_12/from_html/-BYU-Pathway-Worldwide-Website.md',
 '../../data/data_09_12_24/out_sep_12/from_html/-Common-Misconceptions-about-Choosing-Certificates.md',
 '../../data/data_09_12_24/out_sep_12/from_html/-Communication-Resources.md']

In [10]:
len(files_list)


601

In [11]:
documents = []

for i, filepath in enumerate(files_list):
    with open(filepath, "r", encoding="utf-8") as file:

        ## Hay metadata dentro de --- ---, necesito obtener el url: que hay dentro de los ---
        readed = file.read()
        url = ""
        title = ""
        subheading = ""
        heading = ""

        # split the metadata from the text
        try:
            metadata, text = readed.split("---\n", 2)[1:]
            # get the url from the metadata
            # get url only if exist
            if "url: " in metadata:
                url = metadata.split("url: ")[1].split("\n")[0]

            if "title: " not in metadata:
                title = metadata.split("title: ")[1].split("\n")[0]
            # get subheading
            if "subheading: " in metadata:
                subheading = metadata.split("subheading: ")[1].split("\n")[0]

            # get heading
            if "heading: " in metadata:
                heading = metadata.split("heading: ")[1].split("\n")[0]
        except:
            print(f"not enough values in file {filepath}. Something went wrong.")
            # continue

        # Verify if the text exceeds the maximum number of tokens allowed
        tokens = tokenizer.tokenize(readed)

        if len(tokens) > 8000:
            print(f"File {filepath} has more than 8000 tokens. Splitting it.")
            chunks = split_text_by_tokens(readed)
            for chunk in chunks:
                document = Document(
                    text=chunk,
                    metadata={
                        "heading": heading,
                        "subheading": subheading,
                        "url": url,
                        "title": title,
                    },  # , "subheading": subheading
                )
                documents.append(document)
        else:
            document = Document(
                text=readed,
                metadata={
                    "heading": heading,
                    "subheading": subheading,
                    "url": url,
                    "title": title,
                },  # , "subheading": subheading
            )

        # add the document to a single entry list
        documents.append(document)


File ../../data/data_09_12_24/out_sep_12/from_html/Devotionals--Speeches.md has more than 8000 tokens. Splitting it.
File ../../data/data_09_12_24/out_sep_12/from_html/How-do-I-find-old-Pathway-devotionals.md has more than 8000 tokens. Splitting it.
File ../../data/data_09_12_24/out_sep_12/from_html/What-is-the-link-to-the-Missionary-Services-Website.md has more than 8000 tokens. Splitting it.
not enough values in file ../../data/data_09_12_24/out_sep_12/from_pdf/Learn-About-PC-.md. Something went wrong.


In [12]:
# split_nodes = splitter.get_nodes_from_documents(documents)


In [13]:
# run the pipeline to generate nodes
nodes = pipeline.run(documents=documents)
# print('nodes', len(nodes))


In [14]:
# Inserta nodos en el índice
index.insert_nodes(nodes)

print(f"Nodes inserted: {len(nodes)}")


Nodes inserted: 3744


In [15]:
nodes[0]


TextNode(id_='89f4103a-3e40-4b91-b6d5-d1ecf34e4015', embedding=[0.03176712617278099, -0.01862695999443531, -0.006844423245638609, 0.027391761541366577, -0.019485151395201683, -0.013864705339074135, 0.003438038984313607, 0.022791296243667603, -0.012176461517810822, -0.0017286917427554727, 0.009496373124420643, 0.01646038144826889, 3.912857573595829e-05, -0.02971309795975685, -0.017107540741562843, 0.011015793308615685, -0.020737266167998314, 0.03486224263906479, 0.003970890771597624, 0.010825865902006626, -0.011437853798270226, -0.006102299317717552, -0.013069823384284973, 0.00033039465779438615, 0.010931381024420261, 0.004407020751386881, -0.021454770117998123, 0.0016486759996041656, 0.00177265633828938, 0.02121560089290142, 0.014743998646736145, 0.01132530439645052, 0.03247056156396866, -0.008884385228157043, -0.009454167447984219, -0.05754099041223526, 0.013090926222503185, 0.015771014615893364, -0.07349489629268646, 0.042768850922584534, 0.008103571832180023, 0.022791296243667603, 0

## Chunks File


We must create a file with: Link, chunk metadata, chunk text. Saved as `chunks.csv`.


In [16]:
# now, i need to save all chunks in a csv with its respective, url, chunk metadata, and chunk text

# create a dataframe with the chunks
chunks = []

for node in nodes:
    chunks.append(
        {
            "url": node.metadata["url"],
            "chunk_metadata": node.metadata,
            "chunk_text": node.text,
        }
    )


In [17]:
# convert the chunks to a Dataframe
df = pd.DataFrame(chunks)

df.head()


,url,chunk_metadata,chunk_text
0,https://www.byupathway.edu/policies/handbook/2...,"{'heading': '2. Admission and Tuition', 'subhe...",---\nheading: 2. Admission and Tuition\nsubhea...
1,https://www.byupathway.edu/policies/handbook/2...,"{'heading': '2. Admission and Tuition', 'subhe...",* Be a) at least 17 years old with a high scho...
2,https://www.byupathway.edu/policies/handbook/2...,"{'heading': '2. Admission and Tuition', 'subhe...",* Attend a weekly gathering [(see 5.0 Gatherin...
3,https://www.byupathway.edu/policies/handbook/2...,"{'heading': '2. Admission and Tuition', 'subhe...",* Complete an English Placement Assessment (fo...
4,https://www.byupathway.edu/policies/handbook/2...,"{'heading': '2. Admission and Tuition', 'subhe...",* Strive to live by the [CES Honor Code](https...


In [18]:
output_path = f"{datapath}/temporary/chunks.csv"

df.to_csv(output_path, index=False)

df.head()


,url,chunk_metadata,chunk_text
0,https://www.byupathway.edu/policies/handbook/2...,"{'heading': '2. Admission and Tuition', 'subhe...",---\nheading: 2. Admission and Tuition\nsubhea...
1,https://www.byupathway.edu/policies/handbook/2...,"{'heading': '2. Admission and Tuition', 'subhe...",* Be a) at least 17 years old with a high scho...
2,https://www.byupathway.edu/policies/handbook/2...,"{'heading': '2. Admission and Tuition', 'subhe...",* Attend a weekly gathering [(see 5.0 Gatherin...
3,https://www.byupathway.edu/policies/handbook/2...,"{'heading': '2. Admission and Tuition', 'subhe...",* Complete an English Placement Assessment (fo...
4,https://www.byupathway.edu/policies/handbook/2...,"{'heading': '2. Admission and Tuition', 'subhe...",* Strive to live by the [CES Honor Code](https...


## Retrieve the data from CSV


create the file `retrieved_chunks.csv`


In [19]:
query_mode = VectorStoreQueryMode.DEFAULT

top_k = 16
sparse_top_k = 16 * 5

# create a retriever from the index
retriever = index.as_retriever(
    vector_store_query_mode=query_mode,
    similarity_top_k=top_k,
    sparse_top_k=sparse_top_k,
)


In [20]:
df = pd.read_csv(f"{datapath}/temporary/index_single_quotes.csv")

df.head()


,Question,Ideal Answer,Link,Quote
0,How do I know if a student has a scholarship?,Missionaries can see if a student has a schola...,https://missionaries.prod.byu-pathway.psdops.c...,Discounts/Scholarships Tab\nView the scholarsh...
1,How do I know if a student is registered for a...,There is not a way for Missionaries to verify ...,https://missionaries.prod.byu-pathway.psdops.c...,Checking for Institute Registration for Colleg...
2,How do I know if student is member of the church?,Missionaries can see the Church membership sta...,https://missionaries.prod.byu-pathway.psdops.c...,Verify a Learner’s Membership Status\n1. In th...
3,What information should I track for each student?,The most important things Missionaries should ...,https://missionaries.prod.byu-pathway.psdops.c...,Monitoring Student Progress\nTime in Course—ho...
4,What to do If a student has already taken this...,For a student that is in a course that they ha...,https://pathway-missionary.powerappsportals.co...,Link 1:\nWHAT TO WATCH FOR\nWhen you start a n...


In [21]:
# drop the empty questions
df = df.dropna(subset=["Question"])

# cuenta cuantas rows hay
print(f"Number of rows: {len(df)}")


Number of rows: 112


In [22]:
# print rows with empty Quote
df[df["Quote"].isnull()]


,Question,Ideal Answer,Link,Quote
84,How do students contact a tutor?,Students can use the Academic Support Center ....,https://www.byui.edu/student-guide/online-tuto...,NaN
89,How can a student change the name on their Pat...,PathwayConnect missionaries can change a stude...,https://missionaries.prod.byu-pathway.psdops.c...,NaN
91,Is there an exception request link for online ...,Students can petition for exceptions. Petitio...,https://www.byui.edu/student-records/academic-...,NaN
101,How can I give missionaries or a BOM to friend...,How to Download the Book of Mormon App\n\nWhil...,https://www.churchofjesuschrist.org/serve/shar...,NaN
103,What scholarships are available?,Heber J Grant scholarship; Return Missionary ...,https://www.byupathway.edu/tuition-discounts,NaN
107,How can I help students pay tuition?,Follow the instructions below if you feel prom...,https://pathway-missionary.powerappsportals.co...,NaN
111,What to do after the gathering,Remind lead and observing students to remain a...,https://missionaries.prod.byu-pathway.psdops.c...,NaN


In [23]:
## ORIGINAL

import numpy as np

retrieved_chunks_dict = {}  # Para almacenar los chunks recuperados para cada pregunta
error_count = 0

# Procesa cada pregunta y recupera los chunks
for idx, row in df.iterrows():
    question = row["Question"]
    quote = row["Quote"]
    link = row["Link"]
    answer = row["Ideal Answer"]

    # Asegurarse de que quote sea una cadena de texto, si no, omitir la pregunta
    if pd.isna(quote):
        print(f"Skipping question {question} due to missing quote.")
        continue

    quote = str(quote)  # Convertir quote a cadena de texto, si es necesario


    # Recuperar chunks relacionados con la pregunta
    try:
        retrieved_chunks = retriever.retrieve(question)
    except Exception as e:
        error_count += 1
        print(f"Error retrieving chunks for question: {question}")
        print(e)
        continue

    # Almacenar los datos en el diccionario
    retrieved_chunks_dict[question] = {
        "quote": f"{link}\n\n{quote}",
        "normal_quote": quote,
        "answer": answer,
        "retrieved_chunks": '\n\n\n'.join([f'{chunk.metadata['url']}\n{chunk.text}' for chunk in retrieved_chunks]),
        "normal_chunks": [chunk.text for chunk in retrieved_chunks],
    }

print(f"Total errors encountered: {error_count}")


Skipping question How do students contact a tutor?  due to missing quote.
Skipping question How can a student change the name on their PathwayConnect certificate? due to missing quote.
Skipping question Is there an exception request link for online degree students? due to missing quote.
Skipping question How can I give missionaries or a BOM to friends of the church? due to missing quote.
Skipping question What scholarships are available? due to missing quote.
Skipping question How can I help students pay tuition? due to missing quote.
Skipping question What to do after the gathering due to missing quote.
Total errors encountered: 0


In [24]:
def tokenize(text):
    """Función para dividir el texto en palabras y normalizar el texto"""
    return re.findall(r"\w+", text.lower())  # Extrae palabras y convierte a minúsculas


In [25]:
def quote_in_chunk_percentage(quote, chunk_text):
    """Calcula el porcentaje del quote que está presente en el chunk text"""
    quote_tokens = set(tokenize(quote))  # Tokeniza el quote
    chunk_tokens = set(tokenize(chunk_text))  # Tokeniza el chunk

    # How many words are in the chunk
    common_tokens = quote_tokens.intersection(chunk_tokens)

    # calculate % of words in quote that are in chunk
    if len(quote_tokens) == 0:
        return 0
    return len(common_tokens) / len(quote_tokens)


In [26]:
# create a function to calculate the similarity recall between the quote and the chunk text
def similarity_recall(quote, chunk):
    # convert the chunk list to a string
    # chunk_text = " ".join(chunk)
    # imprime el type de chunk
    return quote_in_chunk_percentage(quote, chunk)


In [27]:
output_df = pd.DataFrame.from_dict(retrieved_chunks_dict, orient="index")

# the actual index must be a column
output_df.reset_index(inplace=True)

# Add the titles

# output_df.columns = ["Question", "Quote", "Link", "Retrieved Chunks"]

output_df.head()


,index,quote,normal_quote,answer,retrieved_chunks,normal_chunks
0,How do I know if a student has a scholarship?,https://missionaries.prod.byu-pathway.psdops.c...,Discounts/Scholarships Tab\nView the scholarsh...,Missionaries can see if a student has a schola...,https://sites.google.com/view/bloom-mtl-onboar...,[* If the student wants the scholarship applie...
1,How do I know if a student is registered for a...,https://missionaries.prod.byu-pathway.psdops.c...,Checking for Institute Registration for Colleg...,There is not a way for Missionaries to verify ...,https://missionaries.prod.byu-pathway.psdops.c...,[The best method is to have the student ask th...
2,How do I know if student is member of the church?,https://missionaries.prod.byu-pathway.psdops.c...,Verify a Learner’s Membership Status\n1. In th...,Missionaries can see the Church membership sta...,https://missionaries.prod.byu-pathway.psdops.c...,[Emphasize that BYU-Pathway is owned by The Ch...
3,What information should I track for each student?,https://missionaries.prod.byu-pathway.psdops.c...,Monitoring Student Progress\nTime in Course—ho...,The most important things Missionaries should ...,https://missionaries.prod.byu-pathway.psdops.c...,[---\nheading: PATH\nsubheading: PATH for Path...
4,What to do If a student has already taken this...,https://missionaries.prod.byu-pathway.psdops.c...,How a Student Withdraws From a Program\nPURPOS...,For a student that is in a course that they ha...,https://pathway-missionary.powerappsportals.co...,[---\nheading: PathwayConnect (PC)\nsubheading...


In [28]:
# apply the function to the dataframe, the columns are Quote and "Retrieved Chunks"
output_df["Recall"] = output_df.apply(
    lambda x: similarity_recall(x["normal_quote"], x["retrieved_chunks"]), axis=1
)

# drop columns normal_quote and normal_chunks
output_df.drop(columns=["normal_quote", "normal_chunks"], inplace=True)

output_df.head()


,index,quote,answer,retrieved_chunks,Recall
0,How do I know if a student has a scholarship?,https://missionaries.prod.byu-pathway.psdops.c...,Missionaries can see if a student has a schola...,https://sites.google.com/view/bloom-mtl-onboar...,1.000000
1,How do I know if a student is registered for a...,https://missionaries.prod.byu-pathway.psdops.c...,There is not a way for Missionaries to verify ...,https://missionaries.prod.byu-pathway.psdops.c...,0.986301
2,How do I know if student is member of the church?,https://missionaries.prod.byu-pathway.psdops.c...,Missionaries can see the Church membership sta...,https://missionaries.prod.byu-pathway.psdops.c...,0.729167
3,What information should I track for each student?,https://missionaries.prod.byu-pathway.psdops.c...,The most important things Missionaries should ...,https://missionaries.prod.byu-pathway.psdops.c...,1.000000
4,What to do If a student has already taken this...,https://missionaries.prod.byu-pathway.psdops.c...,For a student that is in a course that they ha...,https://pathway-missionary.powerappsportals.co...,0.800000


In [29]:
# Guardar los resultados en un archivo CSV
# las columnas son: Question,Quote,Quote URL (Link),retrieved chunks,recall

output_path = f"{datapath}/temporary/chunks_recall.csv"

output_df.columns = ["Question", "Quote", "Answer", "Retrieved Chunks", "Recall"]
# ignore the index when saving

# sort by recall
output_df = output_df.sort_values(by="Recall", ascending=False)

output_df.to_csv(output_path, index=False)

output_df.head()


,Question,Quote,Answer,Retrieved Chunks,Recall
0,How do I know if a student has a scholarship?,https://missionaries.prod.byu-pathway.psdops.c...,Missionaries can see if a student has a schola...,https://sites.google.com/view/bloom-mtl-onboar...,1.0
54,How do I get into Zoom?,https://missionaries.prod.byu-pathway.psdops.c...,Open your Zoom Desktop App. If you are not al...,https://missionaries.prod.byu-pathway.psdops.c...,1.0
70,Does BYU-I have a way to download the admissio...,https://www.byui.edu/student-records/enrollmen...,BYU-I has a tool that provides standard enroll...,https://www.byui.edu/student-records/enrollmen...,1.0
67,Is the Hall scholarship different from the Heb...,https://hall-foundation.org/about-scholarship/...,The Hall Foundation scholarship is different f...,https://hall-foundation.org/about-scholarship/...,1.0
66,How often could one access Mentorship Bridge S...,https://sites.google.com/view/bloom-mtl-onboar...,The Mentor Bridge Scholarship was originally o...,https://sites.google.com/view/bloom-mtl-onboar...,1.0
